In [ ]:
import pandas as pd
import numpy as np
import pyodbc
import logging
from datetime import datetime
from dotenv import load_dotenv
import os


load_dotenv(dotenv_path="v.env")
print(os.getenv('tabla'))


[dbo].[whatsapp_rol_no_rol]


In [ ]:
server = os.getenv('server')
database = os.getenv('database')

In [ ]:
carpeta = f'//{server}/stage/{database}/CSV' 

In [2]:
resultado_importacion = {
    'exitos': 0,
    'errores_dict': {},
    'descripcion_errores': {}
}

In [3]:
def conectar_sql(server, database):
    try:
        conn = pyodbc.connect(
            f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
        )
        logging.info("Conexión a SQL establecida")
        return conn
    except Exception as e:
        logging.error(f"Error al conectar a SQL: {e}")
        raise


In [ ]:
def cargar_datos_csv(conn, tabla, carpeta):
    global resultado_importacion
    cursor = conn.cursor()
    
    cursor.execute("SELECT NombreArchivo FROM dbo.archivos_wsp_rol_no_rol_procesados")
    archivos_procesados = {row[0] for row in cursor.fetchall()}


    exitos = 0
    errores_dict = {}
    descripcion_errores = {}
    contador_error = 1

    for archivo in os.listdir(carpeta):
        if archivo.endswith('.csv') and archivo not in archivos_procesados:
            ruta_completa = os.path.join(carpeta, archivo)

            sql = f"""
            BULK INSERT {tabla}
            FROM '{ruta_completa}'
            WITH (
                FIRSTROW = 2,
                FIELDTERMINATOR = '¦',
                ROWTERMINATOR = '0x0a',
                CODEPAGE = '1252'
            );
            """
            try:
                print(f'Cargando {archivo}...')
                cursor.execute(sql)
                conn.commit()
                print(f'{archivo} carga correcta.')


                cursor.execute(
                    "INSERT INTO dbo.archivos_wsp_rol_no_rol_procesados (NombreArchivo) VALUES (?)",
                    archivo
                )
                conn.commit()

                exitos += 1

            except Exception as e:
                print(f'Error importando {archivo}: {e}')
                error_texto = str(e)

                clave_existente = None
                for clave, descripcion in descripcion_errores.items():
                    if descripcion == error_texto:
                        clave_existente = clave
                        break

                if clave_existente:
                    errores_dict[clave_existente].append(archivo)
                else:
                    nueva_clave = f'error_{contador_error}'
                    errores_dict[nueva_clave] = [archivo]
                    descripcion_errores[nueva_clave] = error_texto
                    contador_error += 1

    cursor.close()

    resultado_importacion['exitos'] = exitos
    resultado_importacion['errores_dict'] = errores_dict
    resultado_importacion['descripcion_errores'] = descripcion_errores

In [5]:
def mostrar_resumen():
    global resultado_importacion

    exitos = resultado_importacion['exitos']
    errores_dict = resultado_importacion['errores_dict']
    descripcion_errores = resultado_importacion['descripcion_errores']

    total_errores = sum(len(v) for v in errores_dict.values())

    print("\n---------- RESUMEN ----------")
    print(f"Archivos importados exitosamente: {exitos}")
    print(f"Archivos con errores: {total_errores}")

    if errores_dict:
        print("\nResumen de errores:")
        for clave, archivos in errores_dict.items():
            print(f"\n{clave}:")
            print(f" - Descripción: {descripcion_errores[clave]}")
            print(f" - Archivos con este error:")
            for archivo in archivos:
                print(f"    · {archivo}")

In [6]:
def seleccionar_datos(conn, tabla):
    query = f"SELECT * FROM {tabla} WHERE RUC IN  ('20100153832', '20502375912')"
    df = pd.read_sql(query, conn)
    logging.info(f"{df.shape[0]} registros cargados desde {tabla}")
    return df

In [7]:
def cargar_datos2(conn, tabla):
    query = f"SELECT * FROM {tabla}"
    df = pd.read_sql(query, conn)
    logging.info(f"{df.shape[0]} registros cargados desde {tabla}")
    return df

In [8]:
if __name__ == "__main__":
    conexion = conectar_sql(os.getenv('server'), os.getenv('database'))
    cargar_datos_csv(conexion, os.getenv('tabla'), carpeta)
    conexion.close()

Cargando ne-export-2025.08.20-19.43 (1)_modificado.csv...
ne-export-2025.08.20-19.43 (1)_modificado.csv carga correcta.
Cargando presuspension_sf_emp B1 10.01_modificado.csv...
presuspension_sf_emp B1 10.01_modificado.csv carga correcta.
Cargando presuspension_sf_emp B1 11.07_modificado.csv...
presuspension_sf_emp B1 11.07_modificado.csv carga correcta.
Cargando presuspension_sf_emp B1 13.06_modificado.csv...
presuspension_sf_emp B1 13.06_modificado.csv carga correcta.
Cargando presuspension_sf_emp B1 14.02_modificado.csv...
presuspension_sf_emp B1 14.02_modificado.csv carga correcta.
Cargando presuspension_sf_emp B1 14.04_modificado.csv...
presuspension_sf_emp B1 14.04_modificado.csv carga correcta.
Cargando presuspension_sf_emp B1 15.08_modificado.csv...
presuspension_sf_emp B1 15.08_modificado.csv carga correcta.
Cargando presuspension_sf_emp B1 16.05_modificado.csv...
presuspension_sf_emp B1 16.05_modificado.csv carga correcta.
Cargando presuspension_sf_emp B1 17.01_modificado.csv.

In [9]:
mostrar_resumen()


---------- RESUMEN ----------
Archivos importados exitosamente: 33
Archivos con errores: 0


In [ ]:
#conn = conectar_sql(os.getenv('server'), os.getenv('database'))
#df = cargar_datos(conn, os.getenv('tabla'))
#df = seleccionar_datos(conn, os.getenv('tabla'))
#df
#conn.close()


In [10]:
conn = conectar_sql(os.getenv('server'), os.getenv('database'))
df = cargar_datos(conn, os.getenv('tabla'))
df
conn.close()

C:\Users\BBARTOLOMEE\AppData\Local\Temp\ipykernel_10712\1950544438.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [12]:
df_cambios = df.copy()

In [18]:
lista_nueva = list(df_cambios.columns)

In [20]:
df_cambios.columns

Index(['campaign_name', 'contact_group_name', 'notification_name',
       'notification_start_date', 'notification_end_date',
       'notification_status', 'customer_id', 'session_id', 'contact_id',
       'phone', 'first_name', 'last_name', 'email', 'notification_sent_date',
       'sent', 'delivered', 'read', 'answered', 'error_reason', 'message',
       'conversation', 'notificationInstanceId'],
      dtype='object')

In [25]:
lista_inclusion = ['campaign_name','sent','read','answered','delivered','notification_sent_date']

In [26]:
for i in lista_inclusion:
    lista_nueva.remove(i)


In [27]:
lista_nueva

['contact_group_name',
 'notification_name',
 'notification_start_date',
 'notification_end_date',
 'notification_status',
 'customer_id',
 'session_id',
 'contact_id',
 'phone',
 'first_name',
 'last_name',
 'email',
 'error_reason',
 'message',
 'conversation',
 'notificationInstanceId']

In [32]:
df_cambios.drop(columns=lista_nueva, inplace = True)

In [33]:
df_cambios.describe()

,campaign_name,notification_sent_date,sent,delivered,read,answered
count,357798,357798,357798,357798,357798,357798
unique,3,48241,2,2,2,2
top,Recibo Digital Rol,2025-06-09 12:23:31,1,1,1,0
freq,195903,200,336571,322035,222959,309578


In [35]:
df_cambios[['sent','read','delivered','answered']].describe()

,sent,read,delivered,answered
count,357798,357798,357798,357798
unique,2,2,2,2
top,1,1,1,0
freq,336571,222959,322035,309578
